<a href="https://colab.research.google.com/github/rustammm/charbot-project/blob/master/Generate_text_with_RuGPTs_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate text with RuGPTs in huggingface
How to generate text with pretrained RuGPTs models with huggingface.

This notebook is valid for all RuGPTs models except RuGPT3XL.
## Install env

In [ ]:
!pip3 install transformers==3.5.0

     |████████████████████████████████| 1.3MB 5.5MB/s 
     |████████████████████████████████| 2.9MB 14.4MB/s 
     |████████████████████████████████| 901kB 41.9MB/s 
     |████████████████████████████████| 1.1MB 43.3MB/s 


In [ ]:
!pip3 install torch==1.4.0

     |████████████████████████████████| 753.4MB 22kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101


In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 613, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 613 (delta 68), reused 46 (delta 24), pack-reused 505
Receiving objects: 100% (613/613), 360.66 KiB | 14.43 MiB/s, done.
Resolving deltas: 100% (368/368), done.


In [ ]:
!ls ru-gpts

convert2huggingface.py	  LICENSE		    requirements.txt
examples		  pretrain_gpt3.py	    scripts
generate_samples.py	  pretrain_transformers.py  src
generate_transformers.py  README.md


## Generate

In [ ]:
import numpy as np
import torch

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
def load_tokenizer_and_model(model_name_or_path):
  return GPT2Tokenizer.from_pretrained(model_name_or_path), GPT2LMHeadModel.from_pretrained(model_name_or_path) #.cuda()


def generate(
    model, tok, text,
    do_sample=True, max_length=30, repetition_penalty=5.0,
    top_k=5, top_p=0.95, temperature=1,
    num_beams=None,
    no_repeat_ngram_size=3
    ):
  input_ids = tok.encode(text, return_tensors="pt") # .cuda()
  out = model.generate(
      input_ids, #.cuda(),
      max_length=max_length,
      repetition_penalty=repetition_penalty,
      do_sample=do_sample,
      top_k=top_k, top_p=top_p, temperature=temperature,
      num_beams=num_beams, no_repeat_ngram_size=no_repeat_ngram_size
      )
  return list(map(tok.decode, out))

### RuGPT2Large

In [ ]:
#tok, model = load_tokenizer_and_model("sberbank-ai/rugpt2large")
#generated = generate(model, tok, "Александр Сергеевич Пушкин родился в ", num_beams=10)
#generated[0]

### RuGPT3Small

In [ ]:
tok, model = load_tokenizer_and_model("sberbank-ai/rugpt3small_based_on_gpt2")
generated = generate(model, tok, "Александр Сергеевич Пушкин родился в ", num_beams=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: ignored

In [ ]:
generated[0]

In [ ]:
print(generate(model, tok, "Кто был президонтом США в 2020 году?", num_beams=3)[0])

### RuGPT3Medium

In [ ]:
#tok, model = load_tokenizer_and_model("sberbank-ai/rugpt3medium_based_on_gpt2")
#generated = generate(model, tok, "Александр Сергеевич Пушкин родился в ", num_beams=10)

In [ ]:
#generated[0]

### RuGPT3Large

In [ ]:
#tok, model = load_tokenizer_and_model("sberbank-ai/rugpt3large_based_on_gpt2")
#generated = generate(model, tok, "Александр Сергеевич Пушкин родился в ", num_beams=10)

In [ ]:
#generated[0]

In [ ]:
generate(model, tok, "\xa0— Кто был президонтом США в 2020 году?", num_beams=2)[0]

In [ ]:
generate(model, tok, "\xa0— Как дела?", num_beams=4)[0]

In [ ]:
generate(model, tok, "\xa0— Что делаешь?", num_beams=2)[0]

In [ ]:
generate(model, tok, "\xa0— Покупаешь машину? \xa0— Да. \xa0— Какую? ", num_beams=2)[0]

In [ ]:
print('Как дела?\xa0— спросил я.\n\n—\xa0Нормально,\xa0— ответил он.\xa0— А у тебя?\n\nЯ рассказал ему о том, что произошло со мной за последние несколько дней. Он внимательно выслушал меня и сказал')

In [ ]:
import time
def test(s):
  t = time.time()
  print('##### Generating')
  print(generate(model, tok, s, num_beams=3)[0])
  print('##### Generated, elapsed', time.time() - t)

In [ ]:
test('\xa0— Покупаешь машину?\n\xa0— Да.\n\xa0— Какую?\n\xa0— ')
test('\xa0— Какая столица Франции?\n\xa0— Париж')


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


##### Generating


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 — Покупаешь машину?
 — Да.
 — Какую?
 —  «Мерседес».
  — А что,
##### Generated, elapsed 1.8325591087341309
##### Generating
 — Какая столица Франции?
 — Париж.

— А где же вы живете?

Он пожал плечами и ответил:

##### Generated, elapsed 3.1473190784454346


In [ ]:
test('\xa0— Какая столица Франции?\n\xa0—')


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


##### Generating
 — Какая столица Франции?
 — спросил я.

— Париж, — ответил он. — Париж — это
##### Generated, elapsed 3.304438352584839


In [ ]:
test('\xa0— Как дела?\n\xa0—')
test('\xa0— Что делаешь?\n\xa0—')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


##### Generating


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 — Как дела?
 — Нормально. Все в порядке.

— А как ты думаешь, почему я не могу с тобой
##### Generated, elapsed 3.454949378967285
##### Generating
 — Что делаешь?
 — спросил я.

— Я хочу, чтобы ты знал, что у меня есть для тебя кое-
##### Generated, elapsed 3.407832145690918


In [ ]:
print(generate(model, tok, '\xa0— Что делаешь?\n\xa0—', num_beams=3))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['\xa0— Что делаешь?\n\xa0— спросил я.\n\n—\xa0Ничего,\xa0— ответил он.\xa0— Просто хочу поговорить с тобой о']
